# Thema: Organisations-IDs für wissenschaftliche Einrichtungen anhand von Emailadressen gewinnen

### 1 Fragestellung

Die Aufgabe bestand darin, die Emailadressen den entsprechenden Institutionen zuzuordnen.

Die Datenquellen dafür waren:

-hybrid_publications.csv https://github.com/subugoe/hybrid_oa_dashboard/blob/master/data/hybrid_publications.csv?raw=true

(Tabelle zu hybriden OA-Artikeln mit Angaben zu Lizenz, Verlag, DOI des OA-Artikels, Erscheinungsjahr, jährl. Umfang der Zeitschrift uvm wie Host, Subdomain, Domain und Suffix, erstellt mit dem Analysewerkzeug Hybrid OA Dashboard)

- GRID Open Data Dump https://figshare.com/articles/GRID_release_2018-11-14/7346852

 (Darin sind Tabellen mit Informationen zu Institutionen, die jew. durch ihre Grid-Id identifiziert werden, sowie Akronyme, Adressen, andere Bezeichnungen für die Institutionen, weitere Ids, uvm. wie z.B. eine Datei, in der das Verhältnis der verschiedenen Institutionen zueinander verzeichnet ist, z.B. Institute, die ihrer Universität untergeordnet sind.)

Außerdem wurde die Datei guete_bearbeitet.csv verwendet, die im Rahmen unserer ersten Aufgabe zur Güte von Recall und Precision erstellt wurde.

### 2 Vorgehen

Aufgabe: Matching domains mit GRID Webseiten-URLs

Tipp: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrlame.join.html

Die Grundidee ist, mithilfe der Programmierbibliothek Pandas von Python passende Spalten aus den vielen verschiedenen Datenquellen per „Join“ zusammenzustellen. So soll man Ende die Emailadressen den Institutionen zuordnen können.

In [1]:
#!pip install pandas
import pandas as pd

In [2]:
hybrid_publications = pd.read_csv('hybrid_publications.csv', usecols=['host', 'domain', 'suffix'])
bearbeitet = pd.read_csv('guete_bearbeitet.csv', index_col = 0)
institutes = pd.read_csv('institutes.csv', usecols=['grid_id','name'])
links = pd.read_csv('links.csv')

In [3]:
print(hybrid_publications.head())

             host    domain  suffix
0  bradford.ac.uk  bradford   ac.uk
1         upol.cz      upol      cz
2       uwe.ac.uk       uwe   ac.uk
3        sina.com      sina     com
4     dlmu.edu.cn      dlmu  edu.cn


In [4]:
print(bearbeitet.head())

              email                                            tdm_url  \
0  meduniwien.ac.at  https://api.elsevier.com/content/article/PII:S...   
1        bruker.com  http://link.springer.com/article/10.1007/s1183...   
2          ohsu.edu  https://api.elsevier.com/content/article/PII:S...   
3           cox.net                                                      
4        amc.uva.nl  http://link.springer.com/article/10.1007/s1176...   

                time_stamp    mine_method                           doi  \
0  2018-12-17 16:06:12 UTC   Elsevier XML   10.1016/j.ddtec.2014.03.009   
1  2018-12-18 08:00:29 UTC  Springer HTML     10.1007/s11837-018-2752-0   
2  2018-12-17 16:06:12 UTC   Elsevier XML  10.1016/j.ocemod.2015.12.007   
3  2018-12-16 12:29:19 UTC  PMC OAI (NIH)     10.1007/s13752-016-0244-4   
4  2019-01-04 14:40:57 UTC  Springer HTML     10.1007/s11764-018-0690-z   

   recall  precision comment  
0    True       True     NaN  
1    True       True     NaN  
2    True  

In [5]:
print(institutes.head())

       grid_id                            name
0  grid.1001.0  Australian National University
1  grid.1002.3               Monash University
2  grid.1003.2        University of Queensland
3  grid.1004.5            Macquarie University
4  grid.1005.4                  UNSW Australia


In [6]:
print(links.head())

       grid_id                      link
0  grid.1001.0    http://www.anu.edu.au/
1  grid.1002.3    http://www.monash.edu/
2  grid.1003.2     http://www.uq.edu.au/
3  grid.1004.5         http://mq.edu.au/
4  grid.1005.4  https://www.unsw.edu.au/


Zunächst wurden die Dateien links und institutes" durch "merge" basierend auf der beiden gemeinsamen Spalte "grid_id" zusammengefügt in der neuen Datei name_and_link.

In [7]:
name_and_link = institutes.merge(links, on='grid_id')

In [8]:
print(name_and_link.head())

       grid_id                            name                      link
0  grid.1001.0  Australian National University    http://www.anu.edu.au/
1  grid.1002.3               Monash University    http://www.monash.edu/
2  grid.1003.2        University of Queensland     http://www.uq.edu.au/
3  grid.1004.5            Macquarie University         http://mq.edu.au/
4  grid.1005.4                  UNSW Australia  https://www.unsw.edu.au/


Mithilfe der Funktionen extract_domain und extract_suffix konnten Bestandteile wie "http://", "www" und "/"  aus der Datei herausgefiltert werden. 

In [9]:
def extract_domain(link):
    link = link.replace('http://', '')
    first_position = link.find('www')
    last_position = link.find('/')
    link = link[first_position+4:last_position]
    link = link.split('.')
    domain = link[0]
    return domain

def extract_suffix(link):
    link = link.replace('http://', '')
    first_position = link.find('www')
    last_position = link.find('/')
    link = link[first_position+4:last_position]
    link = link.split('.')
    suffix_list = []
    for element in link:
        suffix_list.append(element)
    suffix_list = suffix_list[1::]
    suffix = ""
    for element in suffix_list:
        suffix = suffix + "." + str(element)
    return suffix[1::]

In [10]:
print(extract_suffix('http://www.kek.ac.de/'))

ac.de


Dann wurden auch name_and_link mit hybrid_publications basierend auf den Spalten "suffix" zusammengefügt zur Datei merge_link_and_host. 

In [11]:
name_and_link['domain'] = name_and_link['link'].apply(extract_domain)
name_and_link['suffix'] = name_and_link['link'].apply(extract_suffix)

In [12]:
merge_link_and_host = hybrid_publications.merge(name_and_link, on=['domain', 'suffix'])

In [13]:
print(merge_link_and_host.head())

             host    domain suffix      grid_id                    name  \
0  bradford.ac.uk  bradford  ac.uk  grid.6268.a  University of Bradford   
1  bradford.ac.uk  bradford  ac.uk  grid.6268.a  University of Bradford   
2  bradford.ac.uk  bradford  ac.uk  grid.6268.a  University of Bradford   
3  bradford.ac.uk  bradford  ac.uk  grid.6268.a  University of Bradford   
4  bradford.ac.uk  bradford  ac.uk  grid.6268.a  University of Bradford   

                                  link  
0  http://www.bradford.ac.uk/external/  
1  http://www.bradford.ac.uk/external/  
2  http://www.bradford.ac.uk/external/  
3  http://www.bradford.ac.uk/external/  
4  http://www.bradford.ac.uk/external/  


Durch "drop.duplicates" wurden dann alle Doppelungen entfernt.

In [14]:
merge_link_and_host = merge_link_and_host.drop_duplicates().reset_index(drop=True)

In [15]:
print(merge_link_and_host.head())

                     host    domain suffix        grid_id  \
0          bradford.ac.uk  bradford  ac.uk    grid.6268.a   
1  student.bradford.ac.uk  bradford  ac.uk    grid.6268.a   
2                 upol.cz      upol     cz  grid.10979.36   
3           oushi.upol.cz      upol     cz  grid.10979.36   
4          optics.upol.cz      upol     cz  grid.10979.36   

                          name                                 link  
0       University of Bradford  http://www.bradford.ac.uk/external/  
1       University of Bradford  http://www.bradford.ac.uk/external/  
2  Palacký University, Olomouc               http://www.upol.cz/en/  
3  Palacký University, Olomouc               http://www.upol.cz/en/  
4  Palacký University, Olomouc               http://www.upol.cz/en/  


In [16]:
merge_link_and_host.to_csv('test.csv')

Anschließend wurde merge_link_and_host mit bearbeitet (Ergebnis der ersten Aufgabe zu Güte von Recall und Precision) zur neuen Datei join_host_and_bearbeitet zusammengefügt.

In [17]:
join_host_and_bearbeitet = pd.merge(bearbeitet, merge_link_and_host, how='left', left_on='email', right_on='host')

In [18]:
print(join_host_and_bearbeitet.head())

              email                                            tdm_url  \
0  meduniwien.ac.at  https://api.elsevier.com/content/article/PII:S...   
1  meduniwien.ac.at  https://api.elsevier.com/content/article/PII:S...   
2        bruker.com  http://link.springer.com/article/10.1007/s1183...   
3          ohsu.edu  https://api.elsevier.com/content/article/PII:S...   
4          ohsu.edu  https://api.elsevier.com/content/article/PII:S...   

                time_stamp    mine_method                           doi  \
0  2018-12-17 16:06:12 UTC   Elsevier XML   10.1016/j.ddtec.2014.03.009   
1  2018-12-17 16:06:12 UTC   Elsevier XML   10.1016/j.ddtec.2014.03.009   
2  2018-12-18 08:00:29 UTC  Springer HTML     10.1007/s11837-018-2752-0   
3  2018-12-17 16:06:12 UTC   Elsevier XML  10.1016/j.ocemod.2015.12.007   
4  2018-12-17 16:06:12 UTC   Elsevier XML  10.1016/j.ocemod.2015.12.007   

   recall  precision comment              host      domain suffix  \
0    True       True     NaN  medun

Schließlich wurden die Werte in dieser Datei noch anhand der Emailadressen sortiert.

In [19]:
join_host_and_bearbeitet = join_host_and_bearbeitet.sort_values(by=['email']).reset_index(drop=True)

In [20]:
join_host_and_bearbeitet.to_csv('merge.csv')

In [21]:
show_result = pd.read_csv('merge.csv', index_col = 0, usecols = ['email','name', 'grid_id'])

### 3 Ergebnisse

Viele E-Mailadressen konnten korrekt zugeordnet werden.

Problematisch für das Vorgehen sind logischerweise vor allem die Fälle, in denen die Forscher keine Emailadressen ihrer Institution angegeben haben, sondern Gmail, Yahoo oder ähnliches.

In [22]:
print(show_result)

                           grid_id                                      name
email                                                                       
                               NaN                                       NaN
                               NaN                                       NaN
                               NaN                                       NaN
                               NaN                                       NaN
                               NaN                                       NaN
126.com                        NaN                                       NaN
Outlook.com                    NaN                                       NaN
aau.in               grid.411373.3             Anand Agricultural University
adm.cgmh.org.tw                NaN                                       NaN
agh.edu.pl             grid.9922.0  AGH University of Science and Technology
amc.nl                         NaN                                       NaN

### 4 Diskussion

Man könnte noch die Angaben in der Datei „relationships.csv“ miteinbeziehen, um Institutionen, die anderen untergeordnet sind und evtl. die gleiche oder erweiterte Emailadressen verwenden, korrekt zuzuordnen. Dazu haben wir noch keine Lösung.

### 5 Fazit

Insgesamt funktioniert das Vorgehen ganz gut, sodass relativ viele Emailadressen korrekt zugeordnet werden können.